In [3]:
from polygon import RESTClient
import os
import requests
import json
import numpy as np
from datetime import datetime
from datetime import date
from datetime import timedelta
from random import randrange
import time
import matplotlib.pyplot as plt
import os

%matplotlib inline

In [4]:
import numpy as np

In [5]:
test = np.load('/media/shuza/HDD_Toshiba/Datasets/snapshots/ATVI/2019.npy') 
print(test)

[[u'59.35' u'58.835' u'59.5539' u'58.615' u'1881167.0'
  u'2019-12-30T03:30:00Z']
 [u'58.85' u'59.08' u'59.355' u'58.65' u'1732381.0'
  u'2019-12-31T03:30:00Z']]


In [ ]:
class Polygon(object):
    def __init__(self,API_KEY):
        self.key = API_KEY
        self.client = RESTClient(self.key)
        self.domain = 'https://api.polygon.io/v2'
    
    def get_news(self,symbol):
        resp = self.client.reference_ticker_news(symbol)
        return resp
    
    def get_ticks(self,params):
        symbol,date,timestamp,timestampLimit,limit = params.values()
        tick_url = f'{self.domain}/ticks/stocks/trades/{symbol}/{date}?timestamp={timestamp}&timestampLimit={timestampLimit}&limit={limit}&apiKey={self.key}'
        print('tick_url',tick_url)
        r = requests.get(tick_url)
        ticks = r.json()
        return ticks
    
    def get_day_trades(self,symbol,start_date,unix_day_start,unix_day_end):
        results = []
#         elapsed_from_start = 0
        start_time = unix_day_start
        while True:
            print('elapsed_from_start',start_time - unix_day_start)
            params = {'symbol':symbol,'date':start_date,'timestamp':start_time,'timestampLimit':unix_day_end,'limit':50000}
            ticks = self.get_ticks(params)
            data = Polygon.store_data(ticks)
            if data.size > 0:
                results.append(data)
            if 'results' in ticks:
                if len(ticks['results']) < 50000:
                    break
                # Get new start
                temp = datetime.fromtimestamp(int(str(int(data[-1,0]))[:-9]))
                start_time = int(1e+9*(temp.timestamp()))
            else:
                break
        if len(results) > 0:
            results = np.concatenate(results)
        else:
            results = np.array([])
        return results
        

    def get_range(self,symbol,start_date,end_date,symbol_path):
        """
        symbol : string
        start_date : string date
        end_date : string date
        """
        time_diff = datetime.strptime(end_date,'%Y-%m-%d') - datetime.strptime(start_date,'%Y-%m-%d')
        print(f'start date {start_date},end date {end_date},time span of {time_diff}')
        results = []
        i = 1
        while start_date != end_date:
            print('date',start_date)
#             start_time = datetime.strptime(start_date,'%Y-%m-%d') + timedelta(hours=9) + timedelta(minutes=30)
            temp_start = datetime.strptime(start_date,'%Y-%m-%d') + timedelta(hours=9) + timedelta(minutes=30)
            temp_end = datetime.strptime(start_date,'%Y-%m-%d') + timedelta(hours=16)
            unix_day_start = int(1e+9*(temp_start.timestamp()))
            unix_day_end = int(1e+9*(temp_end.timestamp()))

            day_trades = self.get_day_trades(symbol,start_date,unix_day_start,unix_day_end)
            print('day_trades.shape',day_trades.shape)
            if day_trades.size > 0:
                np.save(symbol_path+f'/{start_date}',day_trades)
                results.append(day_trades)
            start_date = datetime.strptime(start_date,'%Y-%m-%d')
            start_date = str((start_date + timedelta(days=1)).date())
            
            if i > 100:
                print('loop break')
                break
            i += 1
        return results
    
    @staticmethod
    def store_data(ticks):
        if 'results' in ticks:
            print('num results',len(ticks['results']))
            data = np.empty((len(ticks['results']),3))
            for i,result in enumerate(ticks['results']):
                data[i,0] = result['t']
                data[i,1] = result['s']
                data[i,2] = result['p']
        else:
            data = np.array([])
        return data
    
    def get_symbol_data(self,symbol_list,start,end,num_samples,folder):
        """
        symbol_list : list
        start : datetime object
        end : datetime object
        num_samples : int
        """
        stock_data = {}
        for symbol in symbol_list:
            symbol_results = []
            symbol_path = os.path.join(folder,str(symbol))
            if not os.path.isdir(symbol_path):
                os.mkdir(symbol_path)
            for _ in range(num_samples):
                random_start_date = random_date(start,end)
                start_date = datetime.strptime(str(random_start_date.date()),'%Y-%m-%d')
                
                string_start_date = str(start_date.date())
                string_end_date = str((start_date + timedelta(days=1)).date())
                
                results = self.get_range(symbol,string_start_date,string_end_date,symbol_path)
                symbol_results.append(results)
            # To store data in dict
            # stock_data[symbol] = symbol_results
            # TO store data in file
        return stock_data
    
    def get_ticker_names(self,params):
        """
        number of tickers : 24852
        max perpage : 50
        number of pages needed to traverse entire list : 498
        start_page : 1 - 497
        end_page : 2 - 498
        """
        assert min(max(params['start_page'],1),497) == params['start_page']
        assert min(max(params['end_page'],2),498) == params['end_page']
        market = params["market"]
        locale = params["locale"]
        perpage = params["perpage"]
        active = params["active"]
        symbol_arr = np.array([])
        for page in range(params['start_page'],params['end_page']+1):
            url = f'{self.domain}/reference/tickers?sort=ticker&market={market}&locale={locale}&perpage={perpage}&page={page}&active={active}&apiKey={self.key}'
            r = requests.get(url)
            data = r.json()
            stock_names = [stock['ticker'] for stock in data['tickers']]
            symbol_arr = np.hstack((symbol_arr,np.array(stock_names)))
                
        return symbol_arr
        
    
    def get_snapshot(self,symbol,start_date,end_date):
        """
        symbol : string
        start_date : string date
        end_date : string date
        """
        time_diff = datetime.strptime(end_date,'%Y-%m-%d') - datetime.strptime(start_date,'%Y-%m-%d')
        assert time_diff.days > 0
        print(f'start date {start_date},end date {end_date},time span of {time_diff}')
        results = []
        while start_date != end_date:
            print('start_date',start_date)
            url = f'https://api.polygon.io/v1/open-close/{symbol}/{start_date}?apiKey={self.key}'
            r = requests.get(url)
            data = r.json()
            start_date = str((datetime.strptime(start_date,'%Y-%m-%d') + timedelta(days=1)).date())
            if data['status'] == 'OK':
                results.append(data)
        return results
              
    def get_symbol_snapshots(self,symbol_list,start,end):
        """
        symbol_list : list
        start : date string %Y-%m-%d
        end : date string %Y-%m-%d
        """
        stock_data = {}
        for symbol in symbol_list:
            symbol_results = []
            results = self.get_snapshot(symbol,start,end)
            stock_data[symbol] = results
        return stock_data
        
def random_date(start, end):
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)           
            

In [ ]:
# Insert API key here
API_KEY = ''

In [ ]:
interface = Polygon(API_KEY)

# Get list of tickers to get data for

In [ ]:
params = {
    'perpage': 50,
    'locale': 'us',
    'market': 'STOCKS',
    'active': 'true',
    'start_page': 1,
    'end_page':4
}
tickers = interface.get_ticker_names(params)

# Get ticker data

In [ ]:
# datetime(year, month, day, hour, minute, second, microsecond)
start = datetime(2017, 1, 1, 0, 0, 0, 0)
# datetime(year, month, day, hour, minute, second, microsecond)
end = datetime(2020, 1, 1, 0, 0, 0, 0)
start_date = str((start.date()))
end_date = str(end.date())

In [ ]:
symbol_list = ['TSLA','AAPL']
num_samples = 5
folder = '/Users/morgan/Code/Jupyter_notebooks/market_data'

In [ ]:
results = interface.get_symbol_data(symbol_list,start,end,num_samples,folder)

In [ ]:
print(results)

# Graph results

In [ ]:
def extract_x(data):
    X = np.array([])
    for sample in data:
        print(sample,len(sample[0]))
        if len(sample[0]) == 0:
            continue
        sample_X = sample[0][0][:,2]
        if X.size == 0:
            X = sample_X
        else:
            X = np.hstack((X,sample_X))
    return X

In [ ]:
tesla_results = results['TSLA']

In [ ]:
# print(tesla_results)

In [ ]:
X = extract_x(tesla_results)

In [ ]:
plt.plot(X)

In [ ]:
data = results[0][0]
print(datetime.fromtimestamp(int(str(int(data[0,0]))[:-9])))
print(datetime.fromtimestamp(int(str(int(data[-1,0]))[:-9])))

In [ ]:

string_date = datetime.strptime('2018-02-01','%Y-%m-%d')
print(string_date)
test_date = datetime.fromtimestamp(int(str(int(data[0,0]))[:-9]))
print(test_date)
test_date += timedelta(days=1)
print(test_date)

In [ ]:
resp = interface.get_news('tsla')

In [ ]:
resp.news

In [ ]:
content = []
sources = []
times = []
for i,new in enumerate(news):
#     print(dir(new))
#     print(new.title)
#     print(new.summary)
#     print(new.source)
#     print(new.keywords)
#     print(new.image)
#     print(new.timestamp)
    content.append(new.summary)
    sources.append(new.source)
    times.append(new.timestamp)

In [ ]:
np.unique(sources)

In [ ]:
print(content)

# Tick Data
 
- Timestamp -t
- size -s
- price -p

Sequentially get the tick data up to a certain time.

### Params
- start time
- end time

In [ ]:
start_time = date(2019, 4, 13)
end_time = date(2020,1,1)
# datetime(year, month, day, hour, minute, second, microsecond)
b = datetime(2017, 11, 28, 23, 55, 0, 0)

In [ ]:
data = np.empty((50000,3))
for i,result in enumerate(ticks['results']):
    data[i,0] = result['t']
    data[i,1] = result['s']
    data[i,2] = result['p']

In [ ]:
data[0,0] - data[1,0]

In [ ]:
type(int(data[0,0]))

In [ ]:
len(str(int(data[0,0])))

In [ ]:
test = str(int(data[0,0]))[:-9]

In [ ]:
print(datetime.fromtimestamp(int(str(int(data[0,0]))[:-9])))
print(datetime.fromtimestamp(int(str(int(data[-1,0]))[:-9])))

In [ ]:
datetime.fromtimestamp(int(str(int(data[0,0]))[:-9]))

In [ ]:
len(str(1577944970))

In [ ]:
datetime.fromtimestamp(1577944970)

In [ ]:
print(datetime.utcfromtimestamp(int(data[0,0])))

In [ ]:
dt_object = datetime.utcfromtimestamp(int(data[0,0]))

In [ ]:
data[0:10,:]

In [ ]:
data[-10:,:]

In [ ]:
keys = tickers.keys()

In [ ]:
ticker_names = tickers['tickers']

In [ ]:
names = [name['ticker'] for name in ticker_names]

In [1]:
top_100_nasdaq = [
'ATVI',
'ADBE',
'AMD',
'ALXN',
'ALGN',
'GOOGL',
'GOOG',
'AMZN',
'AAL',
'AMGN',
'ADI',
'ANSS',
'AAPL',
'AMAT',
'ASML',
'ADSK',
'ADP',
'BIDU',
'BIIB',
'BMRN',
'BKNG',
'AVGO',
'CDNS',
'CDW',
'CERN',
'CHTR',
'CHKP',
'CTAS',
'CSCO',
'CTXS',
'CTSH',
'CMCSA',
'CPRT',
'CSGP',
'COST',
'CSX',
'DLTR',
'EBAY',
'EA',
'EXC',
'EXPE',
'FB',
'FAST',
'FISV',
'FOXA',
'FOX',
'GILD',
'IDXX',
'ILMN',
'INCY',
'INTC',
'INTU',
'ISRG',
'JD',
'KLAC',
'KHC',
'LRCX',
'LBTYA',
'LBTYK',
'LULU',
'MAR',
'MXIM',
'MELI',
'MCHP',
'MU',
'MSFT',
'MDLZ',
'MNST',
'NTAP',
'NTES',
'NFLX',
'NVDA',
'NXPI',
'ORLY',
'PCAR',
'PAYX',
'PYPL',
'PEP',
'QCOM',
'REGN',
'ROST',
'SGEN',
'SIRI',
'SWKS',
'SPLK',
'SBUX',
'SNPS',
'TMUS',
'TTWO',
'TSLA',
'TXN',
'TCOM',
'ULTA',
'UAL',
'VRSN',
'VRSK',
'VRTX',
'WBA',
'WDAY',
'WDC',
'WLTW',
'XEL',
'XLNX']

In [2]:
np.save('top_100.npy',top_100_nasdaq)

NameError: name 'np' is not defined